In [45]:
import os
import json
from datetime import datetime
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, List, Dict
from typing_extensions import TypedDict

In [46]:
def get_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Always format your final answer as a JSON string."
        f"\n{suffix}"
    )


In [47]:
class State(TypedDict):
    messages: Annotated[List, add_messages]
    trip_start_date: str
    trip_end_date: str
    traveler_preferences: Dict
    start_flight: str
    return_flight: str
    

    flight_info: Dict
    total_schedule: Dict 
    report_markdown: str
    city: str

In [48]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

In [49]:
def calculate_trip_days(start_date: str, end_date: str) -> int:
    """Calculate the number of days in the trip, including start and end dates."""
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    return (end - start).days + 1

In [ ]:
def _extract_json_object(text: str):
    import re
    import json

    # Try code-fence wrapped JSON
    if isinstance(text, dict):
        return text
    if not isinstance(text, str):
        raise json.JSONDecodeError("Non-string content", str(text), 0)

    fenced = re.search(r"```(?:json)?\s*([\s\S]*?)```", text, re.IGNORECASE)
    if fenced:
        candidate = fenced.group(1).strip()
        try:
            return json.loads(candidate)
        except Exception as e:
            print("Failed to parse fenced JSON:", e)
            print("Candidate was:", candidate[:500])  # Print first 500 chars for debug

    # Greedy braces fallback
    first = text.find("{")
    last = text.rfind("}")
    if first != -1 and last != -1 and last > first:
        candidate = text[first:last + 1]
        try:
            return json.loads(candidate)
        except Exception as e:
            print("Failed to parse greedy braces JSON:", e)
            print("Candidate was:", candidate[:500])

    # Direct parse
    try:
        return json.loads(text)
    except Exception as e:
        print("Failed to parse direct JSON:", e)
        print("Text was:", text[:500])
        raise

In [51]:
def daily_schedule_node(state: State):
    trip_start = state.get("trip_start_date")
    trip_end = state.get("trip_end_date")
    preferences = state.get("traveler_preferences", {})
    flight_info = state.get("flight_info", {})
    city = state.get("city", "")

    # Calculate number of trip days
    num_days = calculate_trip_days(trip_start, trip_end)

    # Build prompts that force a strict JSON-only response
    system_prompt = get_system_prompt(
        "Return ONLY a JSON object following required_output_format. Do not include any prose, code fences, or explanations."
    )

    # Updated user prompt with detailed instructions
    user_prompt = json.dumps(
        {
            "instructions": (
                "Generate a diverse activity pool for the specified city and travel dates. "
                "Food activities should include local restaurants, street food, or cooking classes. "
                "Culture activities should include museums, historical sites, or festivals. "
                "Light hiking should include scenic trails or parks suitable for light outdoor activity. "
                "Tailor activities to the city, travel dates (considering seasonal factors), and preferences. "
                "Schedule meals within lunch (12:00-13:00) and dinner (18:00-19:00) windows. "
                "If preferences are missing, include a balanced mix of food, culture, and light hiking. "
                "For multi-city trips, include transitions between cities based on flight_info. "
                "Each day should have a balanced schedule from 09:00 to 20:00, respecting meal times. "
                "On the first day, account for the start flight arrival time and limit activities to after arrival. "
                "On the last day, account for the return flight departure time and schedule activities before departure. "
                f"Generate a schedule for {num_days} days, labeling them as day1, day2, ..., day{num_days}. "
                "Each day should have 3-5 activities, including at least one meal (lunch or dinner) and one activity from each preference (food, culture, light hiking) where possible."
            ),
            "trip": {
                "city": city,
                "start_date": trip_start,
                "end_date": trip_end,
                "preferences": preferences,
            },
            "flight_info": flight_info,
            "assumptions": {
                "default_open": "09:00-20:00",
                "avg_transit_min": 20,
                "lunch_window": "12:00-13:00",
                "dinner_window": "18:00-19:00",
            },
            "required_output_format": {
                f"day{n}": {
                    "activities": [
                        {
                            "name": "string",
                            "location": "string",
                            "activity_type": "food|culture|light_hiking|meal|transit|rest",
                            "start_time": "HH:MM",
                            "end_time": "HH:MM",
                            "notes": "string"
                        }
                    ]
                } for n in range(1, num_days + 1)
            },
        }
    )
    
    messages = [SystemMessage(content=system_prompt), HumanMessage(content=user_prompt)]

    schedule_content =llm.invoke(messages).content
    try:
        parsed = _extract_json_object(schedule_content)
        state["total_schedule"] = parsed
    except Exception:
        state["total_schedule"] = {f"day{n}": {"activities": []} for n in range(1, num_days + 1)}
        state.setdefault("messages", []).append(SystemMessage(content="Failed to parse schedule JSON"))

    return state,schedule_content

In [52]:
flight_info = {
  "start_flight": {
    "departure_airport": "Hong Kong International Airport",
    "arrival_airport": "Tokyo Narita International Airport",
    "departure_region": "Hong Kong",
    "arrival_region": "Tokyo",
    "departure_time": "10:46",
    "estimated_arrival_time": "14:55"
  },
  "return_flight": {
    "departure_airport": "Tokyo Narita International Airport",
    "arrival_airport": "Hong Kong International Airport",
    "departure_region": "Tokyo",
    "arrival_region": "Hong Kong",
    "departure_time": "16:49",
    "estimated_arrival_time": "21:03"
  }
}

In [53]:
# Test runner
state = State(
    messages=[],
    trip_start_date="2025-03-10",
    trip_end_date="2025-03-15",
    traveler_preferences={"pace": "moderate", "interests": ["food", "culture", "light hiking"]},
    start_flight="UO870",
    return_flight="UO871",
    flight_info=flight_info,
    city="Osaka",
)

out,schedule_content = daily_schedule_node(state)
print("Keys in result:", list(out.keys()))
print("Top-level keys in total_schedule:", list(out["total_schedule"].keys())[:5])

# Validate new schema: expect day1 present with activities list
ts = out.get("total_schedule", {})
assert isinstance(ts, dict)
assert any(k.startswith("day") for k in ts.keys()), "Expected dayN keys in total_schedule"
assert isinstance(ts.get("day1", {}).get("activities", []), list), "day1.activities should be a list"

print("OK: daily_schedule_node (dayN schema) isolated test passed")

Keys in result: ['messages', 'trip_start_date', 'trip_end_date', 'traveler_preferences', 'start_flight', 'return_flight', 'flight_info', 'city', 'total_schedule']
Top-level keys in total_schedule: ['day1', 'day2', 'day3', 'day4', 'day5']
OK: daily_schedule_node (dayN schema) isolated test passed


In [54]:
out

{'messages': [],
 'trip_start_date': '2025-03-10',
 'trip_end_date': '2025-03-15',
 'traveler_preferences': {'pace': 'moderate',
  'interests': ['food', 'culture', 'light hiking']},
 'start_flight': 'UO870',
 'return_flight': 'UO871',
 'flight_info': {'start_flight': {'departure_airport': 'Hong Kong International Airport',
   'arrival_airport': 'Tokyo Narita International Airport',
   'departure_region': 'Hong Kong',
   'arrival_region': 'Tokyo',
   'departure_time': '10:46',
   'estimated_arrival_time': '14:55'},
  'return_flight': {'departure_airport': 'Tokyo Narita International Airport',
   'arrival_airport': 'Hong Kong International Airport',
   'departure_region': 'Tokyo',
   'arrival_region': 'Hong Kong',
   'departure_time': '16:49',
   'estimated_arrival_time': '21:03'}},
 'city': 'Osaka',
 'total_schedule': {'day1': {'activities': [{'name': 'Flight from Hong Kong to Tokyo',
     'location': 'Tokyo Narita International Airport',
     'activity_type': 'transit',
     'start_tim

In [55]:
schedule_content

'```json\n{\n    "day1": {\n        "activities": [\n            {\n                "name": "Flight from Hong Kong to Tokyo",\n                "location": "Tokyo Narita International Airport",\n                "activity_type": "transit",\n                "start_time": "10:46",\n                "end_time": "14:55",\n                "notes": "International flight arrival"\n            },\n            {\n                "name": "Train to Osaka",\n                "location": "Shinkansen to Osaka",\n                "activity_type": "transit",\n                "start_time": "15:30",\n                "end_time": "18:30",\n                "notes": "Take Shinkansen from Tokyo to Osaka"\n            },\n            {\n                "name": "Dinner at Dotonbori",\n                "location": "Dotonbori",\n                "activity_type": "meal",\n                "start_time": "18:45",\n                "end_time": "19:45",\n                "notes": "First taste of Osaka street food in famous foo